In [10]:
"""
tensor completion solver with split conic solver (SCS)
"""
from math_utils import *
import numpy as np
from scipy import linalg
from sktensor import ktensor
import matplotlib.pyplot as plt
import cPickle as pickle
import sys
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
import scs 
from sys import getrefcount
from guppy import hpy
from scipy import sparse

In [11]:
def gen_mask(shape,obv_ratio):
    '''
    Return a mask matrix, 1 for observed, 0 for missing. 
    Args:
        obv_ratio: (0,1) observation ratio
        N: size of the mask
    '''
    np.random.seed()
    obv_idx = np.random.choice([0, 1], size=shape, p=[1.0-obv_ratio, obv_ratio])
    Omega = np.reshape(obv_idx, shape)
    return Omega 

In [12]:
def gen_orth_tensor(N,R):
    # all orthogonal cases
    U = np.random.random((N, R))
    U_orth = linalg.orth(U)

    V = np.random.random((N, R))
    V_orth = linalg.orth(V)

    W = np.random.random((N, R))
    W_orth = linalg.orth(W)

    Lambda = np.random.random((R,))

    X = ktensor([U_orth, V_orth, W_orth], lmbda=Lambda)
    X_ten = np.asarray(X.totensor())

    return X_ten

In [13]:
def eval_completion(X_pred, X_true, Omega):
    """Evaluate the predictio accuracy (elementwise) for the completed entries"""
    Y_pred = X_pred[Omega==0] # return vectors
    Y_true = X_true[Omega==0]
    return linalg.norm(Y_pred - Y_true)/linalg.norm(Y_true)

In [14]:
# Solve each slices separately and average
import sys
from numpy import linalg as LA
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
from cvxpy import *
from math_utils import *

def matrix_recovery(Omega, X):
    """Solve the matrix recovery problem"""
    ndim = np.ndim(X)
    n_rows, n_cols = X.shape
    X_opt = Variable(n_rows, n_cols)
    obj = Minimize(norm(X_opt, "nuc"))
    constraints = [mul_elemwise(Omega, X_opt) == mul_elemwise(Omega, X)]
    prob = Problem(obj, constraints)
    # Use SCS to solve the problem.
    prob.solve(verbose=False, solver=SCS) 
    return np.asarray(X_opt.value)

def tensor_recovery_unfold(Omega, X):
    """ Average of the unfolded tensor recovery problem"""
    ndim = np.ndim(X)
    X_modes = [np.array(X.shape)]*3 # three sets of tensors

    for mode in range(ndim):
        Omega_ufd = unfold(np.copy(Omega), mode)
        X_ufd = unfold(np.copy(X), mode)
        X_mode = matrix_recovery(Omega_ufd, X_ufd)
        X_modes[mode]= fold(X_mode, mode, X.shape)
    X_out = np.mean(np.array(X_modes), axis = 0)#1./3* np.add(np.add(fold(X_out_1, 0, shape), fold(X_out_2, 1, shape) ), fold(X_out_3, 2,shape))
    return X_out


In [15]:
import sys
from numpy import linalg as LA
sys.path.append('/afs/cs.stanford.edu/u/yuqirose/cvxpy')
from cvxpy import *
from math_utils import *


def tensor_recovery(Omega,X):
    """formulate a SDP problem with tensor n-rank nuclear norm minimization problem"""
    shape = X.shape
    n_elems = np.prod(shape)
    X_opt_1 = Variable(shape[0], n_elems / shape[0])
    X_opt_2 = Variable(shape[1], n_elems / shape[1])
    X_opt_3 = Variable(shape[2], n_elems / shape[2])

    # mask   
    Omega_1 = unfold(Omega,0)
    Omega_2 = unfold(Omega,1)
    Omega_3 = unfold(Omega,2)

    # measurements
    X_1 = unfold(X,0)
    X_2 = unfold(X,1)
    X_3 = unfold(X,2)

    obj = Minimize(norm(X_opt_1, "nuc")+ norm(X_opt_2, "nuc") + norm(X_opt_3, "nuc") ) # tensor norm as algebraic mean of matrix norm
    constraints = [mul_elemwise(Omega_1, X_opt_1) == mul_elemwise(Omega_1, X_1),
                  mul_elemwise(Omega_2, X_opt_2) == mul_elemwise(Omega_2, X_2),
                  mul_elemwise(Omega_3, X_opt_3) == mul_elemwise(Omega_3, X_3)]
    #               vec(X_opt_1)== vec((vstack(X_opt_2[:,0:shape[0]],X_opt_2[:,shape[0]:2*shape[0]],
    #                                          X_opt_2[:,2*shape[0]:3*shape[0]], X_opt_2[:,3*shape[0]:4*shape[0]])).T)] # vector format of the variables are the same

    prob = Problem(obj, constraints)
    # Use SCS to solve the problem.
    prob.solve(verbose=False, solver=SCS) 
    X_out_1  = np.asarray(X_opt_1.value)
    X_out_2 = np.asarray(X_opt_2.value)
    X_out_3 = np.asarray(X_opt_3.value)

    X_out = fold(X_out_1,0,shape)#1./3* np.add(np.add(fold(X_out_1, 0, shape), fold(X_out_2, 1, shape) ), fold(X_out_3, 2,shape))
    return X_out

In [16]:
# Setting for the experiments   
# n_rows = 1;
# n_cols = 1;
# num_exp = 5
# N = 20; # size
# max_rank = N

# # obv_ratio = 0.8;
# # succ_thres = 1e-3;
# ten_recv_prob = np.zeros((n_rows, n_cols))

# dgr_range = np.linspace(0.1, 1, n_rows)
# obv_range = np.linspace(0,1,n_cols)   

In [17]:
# from joblib import Parallel, delayed
# import multiprocessing

# inputs = range(10) 
# def ten_rand_exp(X, obv_ratio, exp_id):
#     succ_thres = 1.0e-3
#     N = len(X)
#     Omega = gen_mask((N,N,N),obv_ratio)
#     X_obv = np.copy(Omega)
    
#     if tensor_recovery(Omega, X, succ_thres):
#         return 1
#     else:
#         return 0
   
    
# num_cores = multiprocessing.cpu_count()
# """degree of freedom funciton """
# dgr_func = lambda n, d: np.ceil(n -  np.sqrt(n*n - n*n*d)) # n^2 number of measurements

# for i in range(n_rows):
#     dgr = dgr_range[i]
#     R = dgr_func(N,dgr)
#     X_ten = gen_orth_tensor (N,R)
#     print "data generated"
#     for j in range(n_cols):
#         obv_ratio = obv_range[j]
#         result = Parallel(n_jobs=num_cores)(delayed(ten_rand_exp)(X_ten,obv_ratio, exp_id) 
#                                              for exp_id in range(num_exp))
#         succ_rate = 1.0* np.sum(np.asarray(result))/num_exp
#         print('rank', R , 'obv_ratio', obv_ratio, 'succ_rate', succ_rate)
#         ten_recv_prob[i,j] = succ_rate
        
# output_data ={'xticks':obv_range, 'yticks':dgr_range, 'values':ten_recv_prob}
# pickle.dump( output_data, open( "result/ten_recv_50.pkl", "wb" ) )

In [18]:
N = 20
R = 1
obv_ratio = 0.8
X_ten = gen_orth_tensor(N,R)
Omega = gen_mask((N,N,N),obv_ratio)
X_obv = np.copy(X_ten)
X_obv[Omega==0] = 0
# succ_thres = 1e-3

In [19]:
out_ten = tensor_recovery(Omega, X_obv)
print 'tensor completion:', eval_completion(out_ten, X_ten, Omega)

tensor completion: 0.000410712368256


In [20]:
out_ten_unfold = tensor_recovery_unfold(Omega, X_obv)

#out_ten = tensor_recovery(Omega, X_obv)
print 'tensor unfolding completion:', eval_completion(out_ten_unfold,X_ten,Omega)
#print 'tensor completion:', eval_completion(out_ten, X_ten, Omega)

tensor unfolding completion: 0.00178641969315


In [21]:
def tensor_recovery_slice(Omega,X):
    shape = X.shape
    n_elems = np.prod(shape)
    ndim = np.ndim(X)
    
    X_modes = [np.array(X.shape)]*3 # three sets of tensors
    for mode in range(ndim):
        perm_order = np.roll(np.arange(ndim),mode)
        X_tp = np.transpose(X,perm_order)
        Omega_tp =  np.transpose(Omega, perm_order)
        X_mode = np.zeros(X_tp.shape)
        for i in range(X_tp.shape[-1]):
            X_mode[:,:,i]= matrix_recovery(Omega_tp[:,:,i], X_tp[:,:,i])
        perm_order_reverse = np.argsort(perm_order)
        X_modes[mode] = np.transpose(X_mode, perm_order_reverse) 
        X_mode = np.copy(X_modes[mode])
 
    X_out = np.mean(np.array(X_modes), axis = 0)#1./3* np.add(np.add(fold(X_out_1, 0, shape), fold(X_out_2, 1, shape) ), fold(X_out_3, 2,shape))
    return X_out

out_ten_slice= tensor_recovery_slice(Omega, X_obv)
print 'tensor slice-wise completion:', eval_completion(out_ten_slice, X_ten, Omega)

tensor slice-wise completion: 0.00417579238062


In [22]:
for mode in range(3):
    out_mat_mode = matrix_recovery(unfold(Omega, mode), unfold(X_obv, mode))
    print "matrix mode {} completion:".format(mode), eval_completion(out_mat_mode,unfold(X_ten,mode),unfold(Omega,mode)  ) 

matrix mode 0 completion: 0.00313264266851
matrix mode 1 completion: 0.00106380311713
matrix mode 2 completion: 0.00265654805544


In [23]:
X_obv[Omega==0] = np.median(X_obv)

for mode in range(3):
    out_mat_mode = matrix_recovery(unfold(Omega, mode), unfold(X_obv, mode))
    print "matrix mode {} completion:".format(mode), eval_completion(out_mat_mode,unfold(X_ten,mode),unfold(Omega,mode)  ) 

matrix mode 0 completion: 0.00313264268413
matrix mode 1 completion: 0.00106380313469
matrix mode 2 completion: 0.00265654805544
